# Semestrální práce KMA/NA

Jan Půlpán

## Úkol 7.

In [1]:
import numpy as np
import numpy.linalg as la
from scipy.linalg import lu, hilbert
import pandas as pd

results = []

### 7. Řešení soustavy lineárních rovnic

Soustavu řešíme pro dvě zadané regulární matice $A \in \mathbb{R}^{n \times n}$. Vektor přesného řešení $x$ zvolíme náhodně a příslušný vektor pravé strany dopočteme jako $b = Ax$. U matice stanovíme číslo podmíněnosti $\kappa(A)$ a soustavu řešíme pomocí 4 různých metod:
- $LU$ rozkladu s částečnou pivotací,
- $LU$ rozkladu s částečnou pivotací a iteračním zpřesněním,
- $QR$ rozkladu,
- singulárního rozkladu.

Kvalitu vypočteného řešení hodnotíme pomocí následujících charakteristik:
- relativní norma rezidua,
- relativní norma chyby,
- velikost normované relativní zpětné vazby,
- velikost růstového faktoru (v případě $LU$ rozkladu).

Výsledné charakteristiky průběžně zaznamenáváme. Shrnutí a okomentované výsledky jsou na konci notebooku.


In [2]:
def rel_sol_err(x_num, A, b):
    return la.norm(b - (A@x_num)) / la.norm(b)

def rel_err(x, x_num):
    return la.norm(x - x_num) / la.norm(x)

def rel_back_err(x, x_num, A, b):
    return la.norm(b - (A@x_num)) / (la.norm(A,ord=2) * la.norm(x) + la.norm(b))

def growth_factor(A, U):
    return la.norm(U, ord=np.inf) / la.norm(A, ord=np.inf)

#### I. Matice $A \in \mathbb{R}^{80 \times 80}$ s 1 na diagonále, -1 pod diagonálou a 1 v posledním sloupci

Nejprve sestavíme matici $A$, vygenerujme náhodný vektor $x$ jako přesné řešení a k němu dopočteme pravou stranu $b$.

In [3]:
n = 80
A = np.array([[-float(k < l) for k in range(n)] for l in range(n)]) + np.eye(n)
A[:-1,-1] = 1

x = np.random.rand(n,1)
b = A @ x


Zjistíme číslo podmíněnosti $\kappa(A)$. 

In [4]:
k_A = la.cond(A)

a) řešení soustavy pomocí $LU$ rozkladu s částečnou pivotací

$Ax = b, PA = LU \rightarrow LUx = Pb$ tu pak řešíme ve 2 krocích
1. rovnice pro $y$: $Ly = Pb$
2. rovnice pro $x$: $Ux = y$

In [5]:
P, L, U = lu(A)
y = la.inv(L) @ P.T @ b
x_num = la.inv(U) @ y

results.append(['LU','A',
               rel_sol_err(x_num, A, b),
               rel_err(x, x_num),
               rel_back_err(x, x_num, A, b),
               growth_factor(A, U)])

b) řešení soustavy pomocí $LU$ rozkladu s částečnou pivotací a iteračním zpřesněním
(používá $LU$ rozklad z předchozího řešení)

In [6]:
iter_n = 2

for i in range(iter_n):
    y = la.inv(L) @ P.T @ (b - A @ x_num)
    e = la.inv(U) @ y
    x_num += e

results.append(['LU iter','A',
               rel_sol_err(x_num, A, b),
               rel_err(x, x_num),
               rel_back_err(x, x_num, A, b),
               ''])

c) řešení soustavy pomocí $QR$ rozkladu

$$\begin{equation}
\begin{aligned}
Ax = QRx &= b\\
x & = R^{-1}Q^Tb
\end{aligned}
\end{equation}$$


In [7]:
Q, R = la.qr(A)
x_num = la.inv(R) @ (Q.T @ b)

results.append(['QR','A',
               rel_sol_err(x_num, A, b),
               rel_err(x, x_num),
               rel_back_err(x, x_num, A, b),
               ''])

d) řešení soustavy pomocí singulárního rozkladu

$$\begin{equation}
\begin{aligned}
Ax = U \Sigma V^*x &= b\\
x & = V\Sigma^{-1}U^* b
\end{aligned}
\end{equation}$$

In [8]:
U, s, Vh = la.svd(A)
x_num = Vh.T @ (np.diag(1/s) @ (U.T @ b))

results.append(['SVD','A',
               rel_sol_err(x_num, A, b),
               rel_err(x, x_num),
               rel_back_err(x, x_num, A, b),
               ''])

#### II. Hilbertova matice

Druhou testovací maticí je Hilbertova matice $A = [a_{i,j}]$, velikosti 10, pro kterou platí $$a_{i,j} = \frac{1}{i+j-1}.$$ Hilbertovu matici $A$ získáme pomocí funkce `scipy.linalg.hilbert`.

In [9]:
n = 10
A = hilbert(n)
x = np.random.rand(n,1)
b = A @ x

Zjistíme číslo podmíněnosti Hilbertovy matice $\kappa(A)$.

In [10]:
k_H = la.cond(A)

a) řešení soustavy pomocí $LU$ rozkladu s částečnou pivotací.

In [11]:
P, L, U = lu(A)
y = la.inv(L) @ P.T @ b
x_num = la.inv(U) @ y

results.append(['LU','Hilbert',
               rel_sol_err(x_num, A, b),
               rel_err(x, x_num),
               rel_back_err(x, x_num, A, b),
               growth_factor(A, U)])

b) řešení soustavy pomocí $LU$ rozkladu s částečnou pivotací a iteračním zpřesněním (používá $LU$ rozklad z předchozího řešení)

In [12]:
iter_n = 2

for i in range(iter_n):
    y = la.inv(L) @ P.T @ (b - A @ x_num)
    e = la.inv(U) @ y
    x_num += e

results.append(['LU iter','Hilbert',
               rel_sol_err(x_num, A, b),
               rel_err(x, x_num),
               rel_back_err(x, x_num, A, b),
               ''])

c) řešení soustavy pomocí $QR$ rozkladu

In [13]:
Q, R = la.qr(A)
x_num = la.inv(R) @ (Q.T @ b)

results.append(['QR','Hilbert',
               rel_sol_err(x_num, A, b),
               rel_err(x, x_num),
               rel_back_err(x, x_num, A, b),
               ''])

d) řešení soustavy pomocí singulárního rozkladu

In [14]:
U, s, Vh = la.svd(A)
x_num = Vh.T @ (np.diag(1/s) @ (U.T @ b))

results.append(['SVD','Hilbert',
               rel_sol_err(x_num, A, b),
               rel_err(x, x_num),
               rel_back_err(x, x_num, A, b),
               ''])

## Výsledky

Nejprve se budeme zabývat vlivem čísla podmíněnosti matice $\kappa(A)$ na výsledné řešení. Matice $A$ má poměrně malé číslo podmíněnosti $\kappa(A) \approx 36$, oproti Hilbertově matici, která má číslo podmíněnosti $\kappa(H) \approx 1.6 \times 10^{13}$. Vysoké číslo podmíněnosti má vliv na relativní normu chyby $\frac{\Vert x - \hat{x} \Vert}{\Vert x \Vert}$ a to až takovou, že všechny použité metody vykazují relativní normu chyby vysoko nad strojovou přesností v řádu $10^{-4}$. Relativní norma rezidua naopak při vysokém číslu podmíněnosti nemusí vypovídat nic o přesnosti aproximace řešení protože platí $$\frac{\Vert x - \hat{x} \Vert}{\Vert x \Vert} \leq \kappa(H)\frac{\Vert b - A\hat{x} \Vert}{\Vert b \Vert}.$$ Pro matici $A$, která má číslo podmíněnosti nízké, nic z tohoto neplatí.

Charakteristika, která ovlivňuje výsledek speciálně při využití $LU$ rozkladu je růstový faktor $\frac{\Vert U \Vert_\infty}{\Vert A \Vert_\infty}$. Pokud je růstový faktor velký a obecně velikost prvků matice $U$ roste exponenciálně v porovnání s prvky matice $A$ ovlivní to významně relativní normu rezidua $\frac{\Vert b - A\hat{x} \Vert}{\Vert b \Vert}$. Růstový faktor je v případě $LU$ rozkladu Hilbertovy matice roven $1$ a norma rezidua je tak na úrovní strojové přesnosti. V případě matice $A$ je ale růstový faktor $\approx 7.5 \times 10^{21}$ a proto i norma rezidua je rovna přibližně $2.8$. $LU$ rozklad je podmíněně stabilní a stabilita závisí právě na růstovém faktoru. Iterační zpřesnění dokáže ovšem tuto nestabilitu potlačit a to již při jedné iteraci navíc, jak je vidět z výsledků v tabulce. 

In [15]:
print(f'Číslo podmíněnosti matice A: {k_A}')
print(f'Číslo podmíněnosti Hilbertovy matice: {k_H}')

Číslo podmíněnosti matice A: 35.8019485047514
Číslo podmíněnosti Hilbertovy matice: 16024416992541.715


In [16]:
df = pd.DataFrame(results, columns = ['Metoda',
                                      'Matice',
                                      'Rel. norma rezidua',
                                      'Rel. norma chyby',
                                      'Rel. zpětná chyba',
                                      'Růstový faktor'])
df

,Metoda,Matice,Rel. norma rezidua,Rel. norma chyby,Rel. zpětná chyba,Růstový faktor
0,LU,A,1.693709e+00,1.189010e+01,7.074720e-01,7.55579e+21
1,LU iter,A,9.248913e-13,1.327282e-11,3.863324e-13,
2,QR,A,5.273145e-16,7.734207e-15,2.202623e-16,
3,SVD,A,1.427529e-15,1.548606e-14,5.962871e-16,
4,LU,Hilbert,1.317911e-15,5.066965e-04,6.232328e-16,1
5,LU iter,Hilbert,1.013178e-16,7.258702e-04,4.791265e-17,
6,QR,Hilbert,4.044262e-16,2.588028e-04,1.912509e-16,
7,SVD,Hilbert,2.433035e-16,2.330009e-04,1.150569e-16,
